In [500]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Metrics & Corss-Validation
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.externals import joblib
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             fbeta_score, make_scorer, classification_report, confusion_matrix)

In [501]:
%store -r full_dataset

In [502]:
full_dataset.keys()

dict_keys(['TBL_TR_Accident_Driver', 'tbl_CR_Victim_Info', 'TBL_TR_Offence_details', 'tbl_TR_Offence', 'TBL_TR_Person_Details', 'TBL_TR_Accident_Casualties', 'tbl_CR_FIR_Details', 'Masters', 'TR_Accident_Vehicle', 'TBL_TR_Offence_Master', 'tbl_tr_accident_report', 'tbl_CR_Arr_Person_Details', 'TR_Accident_Vehicle_Master'])

In [503]:
fir = full_dataset['tbl_CR_FIR_Details']['tbl_CR_FIR_Details']

In [504]:
fir.head()

,FIR_ID,Unit_ID,Crime_No,CCIS_Crime_No,FIR_Entry_Type,Offence_From_Date,Offence_To_Date,FIR_Category,FIR_Date,Delay_Reason,...,Action_Flag,Sync_Id,Sync_Id_Old,DEDT,Insert_KGSC,Update_KGSC,GSC_No,FIR_Submitted_dt,Dispatch_Juvenile_Court_Id,key
0,2014000001,57,10973005720140001,NaN,1,2014-01-01 00:15:00,2014-01-01 00:15:00,2,2014-01-01 01:00:00,NaN,...,N,-1,-1,2014-01-01 01:42:24.687,1,1,PO0057140100001,2014-01-01 01:54:14.520,NaN,572014000001
1,2014000001,58,10973005820140001,NaN,1,2014-01-01 01:30:00,2014-01-01 01:30:00,2,2014-01-01 03:30:00,NaN,...,N,-1,-1,2014-01-01 19:22:16.687,1,1,PO0058140100001,2014-01-02 08:00:01.783,NaN,582014000001
2,2014000001,59,10973005920140001,NaN,1,2013-12-31 19:30:00,2013-12-31 19:30:00,2,2014-01-01 16:30:00,ಗಾಯಾಳುಗಳಿಬ್ಬರಿಗೂ ಆಸ್ಪತ್ರೆಯಲ್ಲಿ ಚಿಕಿತ್ಸೆ ಕೊಡಿಸು...,...,N,-1,-1,2014-01-01 17:24:41.970,1,1,PO0059140100001,2014-01-01 19:38:54.880,NaN,592014000001
3,2014000001,119,10438011920140001,NaN,1,2014-01-01 15:00:00,2014-01-01 15:00:00,2,2014-01-01 16:30:00,NaN,...,N,-1,-1,2014-01-01 19:54:39.833,1,1,PO0119140100001,2014-01-01 20:08:45.137,NaN,1192014000001
4,2014000001,138,10439013820140001,NaN,1,2014-01-05 11:30:00,2014-01-05 11:30:00,2,2014-01-05 13:30:00,No,...,N,-1,-1,2014-01-05 21:11:59.307,1,1,PO0138140100001,2014-01-05 21:15:10.500,NaN,1382014000001


In [505]:
accidents = pd.read_csv('accident_cleaned.csv')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [506]:
fir.columns

Index(['FIR_ID', 'Unit_ID', 'Crime_No', 'CCIS_Crime_No', 'FIR_Entry_Type',
       'Offence_From_Date', 'Offence_To_Date', 'FIR_Category', 'FIR_Date',
       'Delay_Reason', 'Complaint_Mode', 'Info_PS_Date', 'GD_No', 'Address1',
       'Address2', 'City', 'State_ID', 'PIN_Code', 'Beat_ID',
       'Village_Area_ID', 'Direction_Distance', 'Other_PS_State',
       'Other_PS_District', 'Other_PS_Limit_ID', 'UD_Case_No', 'Brief_Facts',
       'Action_Taken', 'Transfered_Unit_ID', 'Transfer_Details',
       'Dispatch_Court_ID', 'Dispatch_Court_Date', 'Dispatch_Officer_ID',
       'Dispatch_Officer_Rank', 'FIR_Type', 'FIR_Status', 'FIR_Stage',
       'UserID', 'FIR_Read_Explained', 'Reason_Decline_Investigation',
       'Occurence_Place_ID', 'Latitude', 'Longitude', 'Request_Status',
       'FIR_Read', 'FIR_Read_Date', 'Copy_To', 'Goonda_Act_case', 'TADA_Case',
       'NSA_Case', 'Case_AgainstPP', 'Firing_Case', 'Lathi_Charge_case',
       'Tear_Gas', 'Arrest_Date', 'Firing_Type', 'Agitation',

In [507]:
accidents.columns

Index(['Unnamed: 0', 'Unit_ID', 'Crime_No', 'NoOfVehicle_Involved',
       'Accident_Road', 'Accident_Description', 'UserID',
       'NoOf_Animal_Injured', 'NoOf_Animal_Killed', 'DEDT', 'NoOfMinorInjured',
       'NoOfNonVehicle', 'RoadNumber', 'SpeedLimit', 'Road_typeb',
       'SteepGradient', 'Footpath', 'Collision_Typeb', 'Fatal_NonFatal_val',
       'Accident_Classification_val', 'Severity_val', 'Collision_Type_val',
       'Accident_Spot_val', 'Accident_Location_val', 'Junction_Control_val',
       'Road_Character_val', 'Road_Type_val', 'Road_Classification_val',
       'Seperation_val', 'Surface_Type_val', 'Surface_Condition_val',
       'Road_Condition_val', 'weather_val', 'Location_Type_val',
       'Main_Cause_val', 'Hit_Run_val', 'Accident_City_val',
       'Accident_Road_val', 'Lane_Type_val', 'Road_Markings_val',
       'Spot_Conditions_val', 'Side_Walk_val', 'Accident_Location_id_val',
       'RoadJunction_val', 'Accident_spotb_val'],
      dtype='object')

In [508]:
accidents_df = accidents.loc[:,['Unit_ID', 'Crime_No','weather_val','Road_Condition_val','Spot_Conditions_val','Surface_Condition_val']]

In [509]:
acc_old = pd.read_csv('accidents_report.csv')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [510]:
acc_old.columns[50:80]

Index(['Inspection_Officer_Name', 'Inspection_Officer_Rank',
       'Inspection_Officer_Code', 'Actual_DateOf_Occurance',
       'FromDate_Of_Occurance', 'ToDate_Of_Occurance', 'Lane_Type',
       'Road_Markings', 'Spot_Conditions', 'Date_Of_Filling',
       'Highway_Patrol_No', 'Aid_Post_No', 'Aid_Post_Distance',
       'Trauma_Care_Centre_No', 'Trauma_Care_Centre_Distance', 'Checked',
       'Signage', 'Side_Walk', 'Modified_Unit_ID', 'Action_Flag', 'Sync_Id',
       'Sync_Id_Old', 'NoOf_Animal_Injured', 'NoOf_Animal_Killed', 'DEDT',
       'Accident_Location_id', 'NoOfMinorInjured', 'NoOfNonVehicle',
       'RoadNumber', 'Chainage'],
      dtype='object')

In [511]:
acc_old = acc_old.loc[:,['Unit_ID', 'Crime_No','Actual_DateOf_Occurance']]

In [512]:
acc_old.head()

,Unit_ID,Crime_No,Actual_DateOf_Occurance
0,857,2015000096,2015-07-04 19:30:00.000
1,613,2015000098,2015-07-09 23:40:00.000
2,1898,2015000101,2015-08-29 12:30:00.000
3,1461,2015000102,2015-04-11 16:30:00.000
4,1911,2015000104,2015-08-09 08:35:00.000


In [513]:
acc_old['key'] = acc_old['Unit_ID'].map(str)+acc_old['Crime_No'].map(str)

In [514]:
accidents_df['key'] = accidents_df['Unit_ID'].map(str)+accidents_df['Crime_No'].map(str)

In [515]:
df_merge = pd.merge(accidents_df, acc_old, on='key')

In [516]:
df_merge.head()

,Unit_ID_x,Crime_No_x,weather_val,Road_Condition_val,Spot_Conditions_val,Surface_Condition_val,key,Unit_ID_y,Crime_No_y,Actual_DateOf_Occurance
0,857,2015000096,Not Applicable,Not Applicable,NaN,Not Applicable,8572015000096,857,2015000096,2015-07-04 19:30:00.000
1,613,2015000098,Fine,Not Applicable,NaN,Dry,6132015000098,613,2015000098,2015-07-09 23:40:00.000
2,1898,2015000101,Clear,Not Applicable,NaN,Dry,18982015000101,1898,2015000101,2015-08-29 12:30:00.000
3,1461,2015000102,Fine,Not Applicable,NaN,Dry,14612015000102,1461,2015000102,2015-04-11 16:30:00.000
4,1911,2015000104,Others,Not Applicable,NaN,Not Applicable,19112015000104,1911,2015000104,2015-08-09 08:35:00.000


In [517]:
fir['key'] = fir['Unit_ID'].map(str)+fir['FIR_ID'].map(str)

In [518]:
fir = fir.loc[:,['key','Latitude','Longitude']]

In [519]:
fir.head()

,key,Latitude,Longitude
0,572014000001,NaN,NaN
1,582014000001,NaN,NaN
2,592014000001,NaN,NaN
3,1192014000001,NaN,NaN
4,1382014000001,NaN,NaN


In [520]:
df_merge_final = pd.merge(df_merge, fir, on='key')

In [521]:
df_merge_final.head()

,Unit_ID_x,Crime_No_x,weather_val,Road_Condition_val,Spot_Conditions_val,Surface_Condition_val,key,Unit_ID_y,Crime_No_y,Actual_DateOf_Occurance,Latitude,Longitude
0,857,2015000096,Not Applicable,Not Applicable,NaN,Not Applicable,8572015000096,857,2015000096,2015-07-04 19:30:00.000,NaN,NaN
1,613,2015000098,Fine,Not Applicable,NaN,Dry,6132015000098,613,2015000098,2015-07-09 23:40:00.000,NaN,NaN
2,1898,2015000101,Clear,Not Applicable,NaN,Dry,18982015000101,1898,2015000101,2015-08-29 12:30:00.000,NaN,NaN
3,1461,2015000102,Fine,Not Applicable,NaN,Dry,14612015000102,1461,2015000102,2015-04-11 16:30:00.000,NaN,NaN
4,1911,2015000104,Others,Not Applicable,NaN,Not Applicable,19112015000104,1911,2015000104,2015-08-09 08:35:00.000,NaN,NaN


In [522]:
df = df_merge_final.loc[:,['key','weather_val','Road_Condition_val','Spot_Conditions_val',
                           'Surface_Condition_val','Actual_DateOf_Occurance','Latitude','Longitude']]

In [523]:
df.shape

(193119, 8)

In [524]:
df = df[np.isfinite(df['Latitude'])]

In [525]:
df.shape

(127913, 8)

In [526]:
df = df[np.isfinite(df['Longitude'])]

In [527]:
df = df[df['Longitude']!=0]

In [528]:
df.shape

(5468, 8)

In [529]:
def swap(row):
    lat =  row['Latitude']
    lon = row['Longitude']
    if lat<1 or lon<1:
        lat= lat*100
        lon = lon*100
    if lat>lon:
        temp = lat
        lat = lon
        lon = temp
    row['Latitude'] = lat
    row['Longitude'] = lon
    return row
    

In [530]:
df = df.apply(swap,axis=1)

In [531]:
df.columns

Index(['key', 'weather_val', 'Road_Condition_val', 'Spot_Conditions_val',
       'Surface_Condition_val', 'Actual_DateOf_Occurance', 'Latitude',
       'Longitude'],
      dtype='object')

In [532]:
road_val = df.Road_Condition_val.unique()

In [533]:
spot_val = df.Spot_Conditions_val.unique()

In [534]:
surface_val = df.Surface_Condition_val.unique()

In [535]:
weather_list = df.weather_val.unique()

In [536]:
df['Date'] = pd.to_datetime(df['Actual_DateOf_Occurance']) 

In [537]:
df['month'] = df['Date'].dt.month_name()

In [538]:
df['weekend'] = np.where((df['Date'].dt.weekday==5)|(df['Date'].dt.weekday==6),1,0)

In [539]:
def get_time(x):
    if x>=4 and x<=8:
        return 'morning'
    if x>8 and x<=12:
        return 'morning_rush'
    if x>12 and x<=16:
        return 'afternoon'
    if x>16 and x<=19:
        return 'evening'
    if x>19 and x<=22:
        return 'evening_rush'
    return "night"

In [540]:
df['time'] = df['Date'].dt.hour.apply(get_time)

In [541]:
df.head()

,key,weather_val,Road_Condition_val,Spot_Conditions_val,Surface_Condition_val,Actual_DateOf_Occurance,Latitude,Longitude,Date,month,weekend,time
86,17792016000032,Fine,No influence on accident,NaN,Not Applicable,2016-02-18 10:30:00.000,13.035114,77.527796,2016-02-18 10:30:00,February,0,morning_rush
179,17822016000033,Clear,Not Applicable,NaN,Dry,2016-02-24 17:00:00.000,13.023613,77.534615,2016-02-24 17:00:00,February,0,evening
337,17792015000209,Fine,No influence on accident,NaN,Not Applicable,2015-12-13 11:30:00.000,13.065017,77.504298,2015-12-13 11:30:00,December,1,morning_rush
379,18962016000093,Others,Not Applicable,NaN,Dry,2016-04-28 17:30:00.000,12.535180,77.383426,2016-04-28 17:30:00,April,0,evening
912,17682015000046,Clear,Not Applicable,NaN,Dry,2015-09-25 05:10:00.000,13.004487,77.341705,2015-09-25 05:10:00,September,0,morning


In [542]:
df.shape

(5468, 12)

In [543]:
import random
from numpy.random import choice
time_val = ['morning','morning_rush','afternoon','evening','evening_rush','night']

In [544]:
neg_sam = []
for i in range(0,50000):
    data = {}
    data['weather_val'] = random.choice(weather_list)
    data['Road_Condition_val'] = random.choice(weather_list)
    data['Spot_Conditions_val'] = random.choice(spot_val)
    data['Surface_Condition_val'] = random.choice(surface_val)
    data['month'] = random.choice(df['month'].unique())
    data['weekend'] = random.choice([0,1])
    data['time'] = random.choice(time_val)
    neg_sam.append(data)

In [545]:
df_neg = pd.DataFrame(neg_sam)

In [546]:
df_neg['accident'] = 0

In [547]:
df['accident'] = 1

In [548]:
df_train = df.drop(['key','Actual_DateOf_Occurance','Latitude','Longitude','Date'],axis=1)

In [549]:
df_train.head()

,weather_val,Road_Condition_val,Spot_Conditions_val,Surface_Condition_val,month,weekend,time,accident
86,Fine,No influence on accident,NaN,Not Applicable,February,0,morning_rush,1
179,Clear,Not Applicable,NaN,Dry,February,0,evening,1
337,Fine,No influence on accident,NaN,Not Applicable,December,1,morning_rush,1
379,Others,Not Applicable,NaN,Dry,April,0,evening,1
912,Clear,Not Applicable,NaN,Dry,September,0,morning,1


In [550]:
df_train = pd.concat([df_train,df_neg])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [551]:
df_train.shape

(55468, 8)

In [552]:
df_train = df_train.sample(frac=1)


In [553]:
Y = df_train['accident']

In [554]:
X = df_train.drop(['accident'],axis=1)

In [555]:
X = X.fillna("Unknown")

In [556]:
X = pd.get_dummies(X, columns=['weather_val', 'Road_Condition_val','Spot_Conditions_val','Surface_Condition_val',
                               'month','time'])

In [557]:
X.head()

,weekend,weather_val_Clear,weather_val_Cloudy,weather_val_Dust Storn,weather_val_Fine,weather_val_Flooding of Slipways/Rivulets,weather_val_Fog / Mist,weather_val_Heavy Rain,weather_val_Light Rain,weather_val_Mist or Fog,...,month_November,month_October,month_September,month_Unknown,time_afternoon,time_evening,time_evening_rush,time_morning,time_morning_rush,time_night
29256,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
40362,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5917,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19087,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
42467,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [558]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [559]:
rf = RandomForestClassifier(n_estimators=10)

In [560]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [561]:
pred = rf.predict(X_test)

In [562]:
print("Accuracy:",accuracy_score(y_test, pred))

Accuracy: 0.995312781683793


In [563]:
for feature in zip(X.columns, rf.feature_importances_):
    print(feature)

('weekend', 0.010004160737228438)
('weather_val_Clear', 0.09488334632379249)
('weather_val_Cloudy', 0.0011231597452781568)
('weather_val_Dust Storn', 0.0020129085429260946)
('weather_val_Fine', 0.05798731046124219)
('weather_val_Flooding of Slipways/Rivulets', 0.0023676527390167268)
('weather_val_Fog / Mist', 0.002245943181515547)
('weather_val_Heavy Rain', 0.002587505006512547)
('weather_val_Light Rain', 0.0018789555571957438)
('weather_val_Mist or Fog', 0.0017044245586743574)
('weather_val_Not Applicable', 0.0032631376188185504)
('weather_val_Others', 0.007077831575131957)
('weather_val_Snow', 0.0020236616313565807)
('weather_val_Very Cold', 0.0025744955698170026)
('weather_val_Very Hot', 0.004938970216482689)
('weather_val_Wind', 0.0028107723235885574)
('Road_Condition_val_Clear', 0.003518868589822825)
('Road_Condition_val_Cloudy', 0.005343171646471456)
('Road_Condition_val_Construction Work / Material', 0.002746299121470195)
('Road_Condition_val_Drainage Ditch', 0.00082110781322440

In [564]:
test = df_train[df_train['accident']==1]

In [565]:
test.shape

(5468, 8)

In [566]:
x_val = test.drop(['accident'],axis=1)

In [567]:
x_val = x_val.fillna("Unknown")

In [568]:
x_val = pd.get_dummies(x_val, columns=['weather_val', 'Road_Condition_val','Spot_Conditions_val','Surface_Condition_val',
                               'month','time'])

In [569]:
pred_val = rf.predict(x_val)

ValueError: Number of features of the model must match the input. Model n_features is 67 and input n_features is 53 